<a href="https://colab.research.google.com/github/paavni-27/automatedFinancialPlanning/blob/main/amex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

loading datasets

In [4]:
df=pd.read_csv('/content/cost_of_living_us.csv')
df.head()

,case_id,state,isMetro,areaname,county,family_member_count,housing_cost,food_cost,transportation_cost,healthcare_cost,other_necessities_cost,childcare_cost,taxes,total_cost,median_family_income
0,1,AL,True,"Montgomery, AL MSA",Autauga County,1p0c,8505.72876,3454.91712,10829.16876,5737.47984,4333.81344,0.0000,6392.94504,39254.0532,73010.414062
1,1,AL,True,"Montgomery, AL MSA",Autauga County,1p1c,12067.50240,5091.70788,11588.19288,8659.55640,6217.45896,6147.8298,7422.07836,57194.3256,73010.414062
2,1,AL,True,"Montgomery, AL MSA",Autauga County,1p2c,12067.50240,7460.20308,12361.77720,11581.63260,7075.65816,15824.6940,9769.56228,76141.0308,73010.414062
3,1,AL,True,"Montgomery, AL MSA",Autauga County,1p3c,15257.15040,9952.23924,13452.18600,14503.70760,9134.35620,18802.1892,13101.70320,94203.5328,73010.414062
4,1,AL,True,"Montgomery, AL MSA",Autauga County,1p4c,15257.15040,12182.21400,13744.59840,17425.78560,9942.36396,18802.1892,13469.21880,100823.5200,73010.414062


exploring the data- checking for missing values

In [5]:
df.shape

(31430, 15)

In [6]:
df.columns

Index(['case_id', 'state', 'isMetro', 'areaname', 'county',
       'family_member_count', 'housing_cost', 'food_cost',
       'transportation_cost', 'healthcare_cost', 'other_necessities_cost',
       'childcare_cost', 'taxes', 'total_cost', 'median_family_income'],
      dtype='object')

In [7]:
df.isnull().sum()

case_id                    0
state                      0
isMetro                    0
areaname                   0
county                     0
family_member_count        0
housing_cost               0
food_cost                  0
transportation_cost        0
healthcare_cost            0
other_necessities_cost     0
childcare_cost             0
taxes                      0
total_cost                 0
median_family_income      10
dtype: int64

missing values only in the 'median_family_income' column.

In [9]:
# Impute missing values with the mean
median_income_mean = df['median_family_income'].mean()
df['median_family_income'].fillna(median_income_mean, inplace=True)

# Verify if there are any missing values left
print(df.isnull().sum())


case_id                   0
state                     0
isMetro                   0
areaname                  0
county                    0
family_member_count       0
housing_cost              0
food_cost                 0
transportation_cost       0
healthcare_cost           0
other_necessities_cost    0
childcare_cost            0
taxes                     0
total_cost                0
median_family_income      0
dtype: int64


In [8]:
duplicate_rows = df.duplicated()
duplicate_rows

0        False
1        False
2        False
3        False
4        False
         ...  
31425    False
31426    False
31427    False
31428    False
31429    False
Length: 31430, dtype: bool

In [21]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [25]:
# Split the data into training and testing sets
X = df.drop(['total_cost', 'case_id', 'areaname', 'county'], axis=1)  # Features
y = df['total_cost'] / df['median_family_income'] * 100  # Percentage of income to invest and spend


data preprocessing - One-Hot Encoding for categorical variables

In [26]:
X = pd.get_dummies(X, columns=['state', 'isMetro'])

In [32]:
# Data Preprocessing - Handle non-numeric values in 'family_member_count'
# Assuming 'family_member_count' contains values like '1p0c', '1p1c', etc.
X['adults'] = X['family_member_count'].apply(lambda x: int(x.split('p')[0]))
X['children'] = X['family_member_count'].apply(lambda x: int(x.split('p')[1][:-1]))  # Remove 'c' from the end
X.drop('family_member_count', axis=1, inplace=True)

In [34]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [35]:
model = LinearRegression()

In [36]:
X_train.shape, y_train.shape

((25144, 63), (25144,))

In [37]:
X_test.shape, y_test.shape

((6286, 63), (6286,))

In [38]:
model.fit(X_train, y_train)

LinearRegression()

In [39]:
model.intercept_

113.26978274745231

In [40]:
model.coef_.round(2)

array([-1.472e+01, -1.472e+01,  0.000e+00,  0.000e+00,  4.063e+01,
        0.000e+00, -0.000e+00, -0.000e+00, -4.810e+00,  2.470e+00,
       -8.600e-01,  1.100e-01, -3.440e+00,  4.520e+00,  6.150e+00,
        4.440e+00, -3.040e+00, -6.640e+00, -2.000e-02,  7.620e+00,
       -1.110e+00,  2.600e-01, -2.360e+00, -2.770e+00, -4.590e+00,
        3.580e+00, -5.100e-01,  3.870e+00,  9.780e+00, -2.220e+00,
       -3.700e-01, -9.000e-01, -3.810e+00,  8.000e-02,  4.130e+00,
       -1.080e+00,  1.420e+00, -3.260e+00,  2.500e-01,  3.950e+00,
        6.680e+00, -4.470e+00,  2.190e+00, -3.470e+00,  5.800e-01,
        7.390e+00, -4.900e-01,  7.200e+00, -5.270e+00, -3.840e+00,
       -6.180e+00, -6.690e+00,  3.730e+00,  3.020e+00, -3.490e+00,
       -3.580e+00, -3.600e-01,  1.650e+00, -5.430e+00, -1.900e-01,
        1.900e-01, -8.200e-01, -7.500e-01])

In [42]:
model.score(X_train, y_train)

0.9325552481360242

In [43]:
model.score(X_test, y_test)

0.9307582362398259

In [44]:
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Mean Squared Error: {mse}")
print(f"R^2 Score: {r2}")

Mean Squared Error: 85.28326530129887
R^2 Score: 0.9307582362398259


In [45]:
y_pred = model.predict(X_test)

In [46]:
predictions_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
print(predictions_df.head())

           Actual   Predicted
30973  101.852762  102.629719
5671    88.614570   92.611749
9704   117.588149  120.155566
15767  127.308262  123.922576
24281  130.847897  123.046777
